In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

Creating the Crew:
The Process class helps to delegate the workflow to the Agents (kind of like a Manager at work)
In the example below, it will run this hierarchically.
manager_llm lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical, #los procesos pueden variar para conseguir mejores resultados. En este caso se declara un manager que delega las tareas
    verbose=True
)

Running the Crew

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is continuously monitoring and analyzing market data for AAPL to identify trends and predict market movements. I should delegate tasks and ask questions to my team members to gather insights and alerts about significant market opportunities or threats for AAPL.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning",
    "context": "We need insights and alerts about significant market opportunities or threats for AAPL. The data analysis should focus on identifying trends and predicting market movements."
}

> Entering new CrewAgentExecutor ch

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies:

Long Call Strategy:
- Potential Risks: The main risk associated with the long call strategy is the potential loss if the market conditions do not align with the expected increase in AAPL's price. There is also the risk of time decay eroding the value of the call options.
- Mitigation Strategies: To mitigate these risks, traders should closely monitor market conditions and AAPL's price movement, set a stop-loss to manage risk, and consider using options with longer expiration dates to allow for more time for the price to move in the desired direction.

Long Call Spread Strategy:
- Potential Risks: The risks with the long call spread strategy include potential loss if AAPL's price does not increase as expected or if the price moves against the position. There is also the risk of time decay affecting the value of the options.
- Mitigation Strategies: Traders can mitigate these risks by carefully selecting the strike prices for the call options, closely monitoring market conditions, setting a stop-loss, and considering the use of options with longer expiration dates.

Short Put Spread Strategy:
- Potential Risks: The main risks of the short put spread strategy include potential losses if AAPL's price decreases significantly or if the price remains stable instead of increasing. There is also the risk of time decay impacting the value of the put options.
- Mitigation Strategies: To mitigate these risks, traders should monitor market conditions closely, set a stop-loss to manage risk, and consider using options with longer expiration dates to allow for more time for the price to move in the desired direction.

Overall, traders should focus on managing risk, closely monitoring market conditions, setting stop-loss orders, and practicing with paper trading or demo accounts before executing the strategies with real capital to increase the chances of success in trading AAPL.